# Update Cell Composition

## Imports

In [ ]:
from kgforge.core import KnowledgeGraphForge

## Setup
Get an authentication token

For now, the [Nexus web application](https://bbp.epfl.ch/nexus/web) can be used to get a token. We are looking for other simpler alternatives.

- Step 1: From the opened web page, click on the login button on the right corner and follow the instructions.

![login-ui](./login-ui.png)

- Step 2: At the end you’ll see a token button on the right corner. Click on it to copy the token.

![login-ui](./copy-token.png)


In [ ]:
TOKEN = ""

In [ ]:
endpoint_staging = "https://staging.nise.bbp.epfl.ch/nexus/v1"
endpoint_prod = "https://bbp.epfl.ch/nexus/v1"
endpoint=endpoint_prod

forge = KnowledgeGraphForge("../forge-config.yml",
                            token=TOKEN,
                            endpoint=endpoint, 
                            bucket="bbp/atlasdatasetrelease")

## Retrieve CellComposition

In [ ]:
# Enter the id of the CellComposition to update
cell_composition_id = "https://bbp.epfl.ch/neurosciencegraph/data/compositions/54818e46-cf8c-4bd6-9b68-34dffbc8a68c"
cell_composition = forge.retrieve(cell_composition_id)
#print(cell_composition)

In [ ]:
# Get the current revision of the CellComposition
rev = cell_composition._store_metadata["_rev"]
print(f"Current CellComposition version: {rev}")

In [ ]:
schemas = ["CellCompositionVolume", "CellCompositionSummary"]
parts = {}
for ipart in range(len(cell_composition.hasPart)):
    part = cell_composition.hasPart[ipart]
    if part.type in schemas:
        parts[part.type] = {"id": part.id, "index": ipart}
if not parts:
    print(f"The CellComposition provided ({cell_composition_id}) has no part with type in the allowed list: {schemas}.")

## Update CellCompositionVolume/Summary

In [ ]:
def check_part(parts, part_type, part_id):
    if not part_id:
        print(f"No id provided for {part_type}")
        return False
    part_type_id = f"{part_type} with id {part_id}"
    if part_type in parts:
        if part_id == parts[part_type]["id"]:
            print(f"The {part_type} id provided ({part_id}) corresponds to the id of the current {part_type} in the CellComposition. Nothing to do.")
            return False
        else:
            print(f"Updating {part_type_id}")
    else:
        print(f"Adding {part_type_id}")
    return True

In [ ]:
def update_cell_composition(cell_composition, parts, part_type, part_id):
    new_part = forge.retrieve(part_id)
    error = f"The {part_type} in the CellComposition will not be updated."
    if not new_part:
        print(f"No resource found with id {part_id}. {error}")
        return None
    new_part = new_part
    if part_type != new_part.type:
        print(f"The id provided ({part_id}) does not correspond to a resource of type {part_type}. {error}")
        return None
              
    cell_composition.hasPart.pop( parts[part_type]["index"] )
    cell_composition.add_parts(new_part)
    return cell_composition

In [ ]:
# Uncomment the part_type that you want to update
part_type = "CellCompositionVolume"
#part_type = "CellCompositionSummary"

# Enter the id of the new CellCompositionVolume/Summary to link
part_id = ""

check = check_part(parts, part_type, part_id)
if check:
    new_cell_composition = update_cell_composition(cell_composition, parts, part_type, part_id)
    forge.update(new_cell_composition)